<a href="https://colab.research.google.com/github/KNmitra/Bhavcopy/blob/master/Killer_Portfolio_by_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os.path
from os import path

if path.exists('/content/Trading_Project') == False:
  os.mkdir('/content/Trading_Project')

os.chdir('/content/Trading_Project')
!pwd
!ls

/content/Trading_Project


In [ ]:
#Installing some usefull library and import tham
#Downloading historical data from Nsepy
import warnings
warnings.filterwarnings('ignore')
try:
    import nsepy
except:
    !pip install -q nsepy
    import nsepy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import date,timedelta
from nsepy import get_history as history_data
plt.style.use('fivethirtyeight')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
stocksymbols = ['TATAMOTORS','DABUR', 'ICICIBANK','WIPRO','BPCL','IRCTC','INFY','RELIANCE']
#startdate = date(2019,10,14)
start_date= date.today()-timedelta(days=365*10)
end_date = date.today()
print(end_date)
print(f"You have {len(stocksymbols)} assets in your porfolio" )

2023-01-09
You have 8 assets in your porfolio


In [ ]:
#fetching data
df = pd.DataFrame()
for i in range (len(stocksymbols)):
    data = history_data(symbol=stocksymbols[i],start =(start_date),end =(end_date))[['Symbol','Close']]
    data.rename(columns={'Close':data['Symbol'][0]},inplace = True)
    data.drop(['Symbol'],axis = 1 , inplace = True)
    if i == 0 :
        df = data
    if i != 0 :
        df = df.join(data)
    
print(df.tail(10))


In [ ]:
#Get the asset/tickers
assets = df.columns

In [ ]:
print(assets)

Index(['TATAMOTORS', 'DABUR', 'ICICIBANK', 'WIPRO', 'BPCL', 'IRCTC', 'INFY',
       'RELIANCE'],
      dtype='object')


In [ ]:
!pip install PyPortfolioOpt

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
#Calculated the expected annualized returns and the annualized sample covarience matrix of the daily asset returns 
mu = expected_returns.mean_historical_return(df)
S =risk_models.sample_cov(df)


In [ ]:
#Optimize for maximal sharp ratio
ef = EfficientFrontier(mu , S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose = True)

OrderedDict([('TATAMOTORS', 0.0), ('DABUR', 0.78454), ('ICICIBANK', 0.0), ('WIPRO', 0.0), ('BPCL', 0.0), ('IRCTC', 0.0), ('INFY', 0.0), ('RELIANCE', 0.21546)])
Expected annual return: 15.3%
Annual volatility: 21.8%
Sharpe Ratio: 0.61


(0.15326620630870577, 0.21827205505783423, 0.6105509304587573)

In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 72.1 MB/s eta 0:00:00


In [ ]:
#Get the discreat allocation of each share perstock
from pypfopt.discrete_allocation import DiscreteAllocation,get_latest_prices

portfolio_val = 15000
latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights,latest_prices,total_portfolio_value = portfolio_val)
allocation,leftover =  da.lp_portfolio()
print('Discrete_Allocation:',allocation)
print('Funds Remaining:',leftover)


Discrete_Allocation: {'DABUR': 21, 'RELIANCE': 1}
Funds Remaining: 862.6499973690206


In [ ]:
import requests
#Create a function to get the companies name
def get_company_name(symbol):
    url = 'http://d.yimg.com/autoc.finance.yahoo.co/autoc?query='+symbol+'&region=1&lang=en'
    result = requests.get(url).json()

    for r in result['ResultSet']['Result']:
        if r['symbol'] == symbol:
            return r['name']

In [ ]:
#Store the company name in a list
company_name = []
for symbol in allocation:
    company_name.append(get_company_name(symbol))

JSONDecodeError: ignored

In [ ]:
#get discreat allocation list

discrete_allocation_list = []
for symbol in allocation:
    discrete_allocation_list.append(allocation.get(symbol))


In [ ]:
#Create a dataframe for portfolio
portfolio_df = pd.DataFrame(columns = ['Company_Symbol','Discrete_Value' + str(portfolio_value)])

In [ ]:
portfolio_df['Company_Symbol'] = allocation
portfolio_df['Discrete_Value'+str(portfolio_value)] = discrete_allocation_list

In [ ]:
portfolio_df

,Company_Symbol,Discrete_Value5000
DABUR,DABUR,21
RELIANCE,RELIANCE,1
